# Basic Example of Flat Rolling with PyRoll

In [ ]:
# only for Google Colab
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/requirements.txt"
!pip install -r "requirements.txt"

Import the core library and a bunch of plugins. We assign `pr` as a shortcut to the PyRolL library to save typing efforts.

In [ ]:
import pandas as pd
import plotly.express as px

import pyroll.basic as pr
import pyroll.export as pe

Import additional libraries and make some basic configuration.

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, format="[%(levelname)s] %(name)s: %(message)s")
logging.getLogger("pyroll").setLevel(logging.INFO)

## Definition of Initial Workpiece (Profile)

With the following class constructor one defines the state of the workpiece that is fed into the first unit of the rolling sequence.
It holds geometrical values as well as material data.

The example case is a square-shaped profile. The used units are standard SI-Units in every case.

In [ ]:
in_profile = pr.Profile.box(
    height=35e-3,
    width=70e-3,
    corner_radius=2e-3,
    temperature=1100 + 273.15,
    strain=0,
    material="C45",
    density=7.5e3,
    specific_heat_capacity=690,
)

You can also plot the profile by using it as follows:

In [ ]:
in_profile

## Simulating Roll Passes

For simulating a roll pass we have to define the shape of the groove, the properties of the roll and of the pass itself.

The groove can be defined by creating a groove object. Here, the `FlatGroove` class is used to create a flat profile and there are no other parameters needed, except for the `usable_width`.
In this case, we define the roll at the same time. For this we need only one additional property, the nominal radius.

In [ ]:
roll1 = pr.Roll(
    groove=pr.FlatGroove(
        usable_width=250e-3,
    ),
    nominal_radius=300e-3 / 2,
)

The roll can now be used to define the passes. The upper and lower working rolls of the pass are equal, so we need only one.
We add the height of the roll gap and the rolling velocity as additional properties.
For the transports we need to add a duration.
Also, we give a label to the steps for human identification.

In [ ]:
sequence = pr.PassSequence(
    [
        pr.RollPass(
            label="Flach Sack I",
            roll=roll1,
            gap=25e-3,
            velocity=0.3,
        ),
        pr.Transport(
            label="I -> II",
            duration=5,
        ),
        pr.RollPass(label="Flach Sack II", roll=roll1, gap=10e-3, velocity=0.3),
        pr.Transport(
            label="II -> III",
            duration=5,
        ),
        pr.RollPass(label="Flach Sack III", roll=roll1, gap=5e-3, velocity=0.3),
    ]
)

To actually simulate the roll pass, we use the `solve` method and give the input profile as an argument.
The state of the output profile is returned.

In [ ]:
sequence.solve(in_profile)

In [ ]:
sequence

## Extracting Result Data

Now we use the function `to_pandas()` from the pyroll_export package to get our results into a flat table for a better overview.

In [ ]:
df_units = pe.to_pandas(sequence)
df_units

We can use the `roll_passes` attribute for filtering out just the passes.

In [ ]:
df_roll_passes = pe.to_pandas(sequence.roll_passes)
df_roll_passes

We can use the `profiles` attribute to extract all profiles.

In [ ]:
df_profiles = pe.to_pandas(sequence.profiles)
df_profiles

## Plotting the Results

Now we can use these tablse with the plotting package `plotly` to show our results in diagrams. We give the data source (the table) and the headers of the columns to display at the respective axes.

### Temperature Evolution over Time

In [ ]:
px.line(df_profiles, x="t", y="temperature", markers=True)

### Roll Forces of Each Pass

In [ ]:
px.bar(df_roll_passes, x="label", y="roll_force", text_auto=".2s")